# Класс Research: эксперименты с пайплайнами. Часть 2
В классе реализована возможность параллельного выполнения экспериментов с использованием нескольких процессов и GPU.

### Пример использования

Усложним ранее рассмотренный пример. Предположим, что у пайплайна достаточно сложный препроцессинг, поэтому для экономии времени один обработанный батч можно пропускать через разные модели.

In [1]:
import sys

sys.path.append('..')

from dataset import B, C, V, Config, Pipeline
from dataset.opensets import MNIST
from dataset.models.tf import VGG7

In [2]:
import tensorflow as tf

model_config = {
    'session/config': tf.ConfigProto(allow_soft_placement = True, 
                                     gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.4, 
                                                               allow_growth=True)),
    'inputs/images': {
        'shape': (28, 28, 1),
        'type': 'float32',
        'name': 'reshaped_images'
    },
    'inputs/labels': {
        'classes': 10,
        'type': 'int32',
        'transform': 'ohe',
        'name': 'targets'
    },
    'input_block/inputs': 'images',
    'output/ops': ['accuracy'],
    'loss': 'ce',
    'optimizer': 'Adam'
}

feed_dict = {'images': B('images'),
             'labels': B('labels')}

In [3]:
pipeline_config = Config(model_config=model_config)

Выделим препроцессинг в отдельный пайплайн:

In [4]:
mnist = MNIST()

train_preproc = mnist.train.p.run(batch_size=32, shuffle=True, n_epochs=None, lazy=True)
test_preproc = mnist.test.p.run(batch_size=100, shuffle=True, n_epochs=None, lazy=True)

vgg7_train = (Pipeline()
              .init_variable('loss', init_on_each_run=list)
              .init_model('dynamic', VGG7, 'model', C('model_config'))
              .train_model('model', feed_dict=feed_dict, fetches='loss', save_to=V('loss'), mode='a')
             )

vgg7_test = (Pipeline()
             .init_variable('accuracy', init_on_each_run=list)
             .import_model('model', C('import_model_from'))
             .predict_model('model', feed_dict=feed_dict, fetches='output_accuracy', save_to=V('accuracy'), mode='a')
            )

ExtractingExtractingExtracting Extracting  C:\Users\kozhevin\AppData\Local\Temp\train-images-idx3-ubyte.gz C:\Users\kozhevin\AppData\Local\Temp\train-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\t10k-images-idx3-ubyte.gz
C:\Users\kozhevin\AppData\Local\Temp\t10k-labels-idx1-ubyte.gz




Отметим, что пайплайны для препроцессинга должны содержать run с параметром lazy=True. Если препроцессинга нет, то lazy run должен быть в самом пайплайне.

In [5]:
from grid import KV, Option
p1 = KV('model_config/body/block/layout', 'layout')
op = Option(p1, ['cna', 'can', 'acn'])

In [6]:
from research import Research

mr = Research()
mr.add_pipeline(vgg7_train, 'loss', preproc=train_preproc, config=pipeline_config, name='train')
mr.add_pipeline(vgg7_test, 'accuracy', preproc=test_preproc, config=pipeline_config, import_model_from='train')
mr.add_grid_config(op)
mr.run(n_reps=10, n_iters=100, n_jobs=2, model_per_preproc=2, name='preproc_research')

<img src="pic1.png">

В таком случае в каждом воркере (парметр n_jobs) на одних и тех же батчах прогоняются сразу два пайплайна с разными конфигами в разных тредах. В данном примере такая возможность не представляет интереса, поскольку весь препроцессинг заключается в загрузке данных, однако в реальных задачах за счёт этого можно выиграть время, особенно если есть несколько GPU.

Параметр model_per_preproc может быть не только числом, но и листом из словарей:

In [7]:
model_per_preproc = [{'model_config/device': 'device:GPU:0'}, {'model_config/device': 'device:GPU:1'}]

В таком случае каждый батч будет обрабатываться двумя пайплайнами. В каждый конфиг будет добавлено по одному элементу списка.

In [8]:
mr = Research()
mr.add_pipeline(vgg7_train, 'loss', preproc=train_preproc, config=pipeline_config, name='train')
mr.add_pipeline(vgg7_test, 'accuracy', preproc=test_preproc, config=pipeline_config, import_model_from='train')
mr.add_grid_config(op)
mr.run(n_reps=10, n_iters=100, n_jobs=2, model_per_preproc=model_per_preproc, name='gpu_research')

Параметр n_jobs также может принимать не только целочисленные значения, но и быть списком экземпляров класса Worker или его наследников. Благодаря этому можно кастомизировать работу воркеров с заданиями.